In [ ]:
import json

with open('/home/pgajo/working/food/data/TASTEset/data/EW-TASTE/EW-TT-PE.json', 'r', encoding='utf8') as f:
    data = json.load(f)
sample = data['annotations'][0]
sample

In [ ]:
import random

def shuffle_entities_ingredient(sample, shuffle_lang, verbose = False) -> dict:
    '''
    Shuffles entities and text for a sample.
    Only shuffles, so the source and target entities need 
    to be linked somehow after shuffling one language
    (usually the target language)!
    '''
    text_key = f'text_{shuffle_lang}'
    ent_key = f'ents_{shuffle_lang}'
    shuffled_ents = []
    shuffled_indexes = [i for i in range(len(sample[ent_key]))]
    random.shuffle(shuffled_indexes)
    sample.update({f'idx_{shuffle_lang}': shuffled_indexes})
    
    # get non-entity positions and strings from the original text
    blanks = []
    for i in range(len(sample[ent_key])-1):
        end_prev = sample[ent_key][i][1]
        start_foll = sample[ent_key][i+1][0]
        blanks.append([end_prev, start_foll, sample[text_key][end_prev:start_foll]])
        
    ent_start = 0
    shuffled_text = ''
    for idx in range(len(sample[ent_key])):
        tmp_ent = sample[ent_key][idx]
        text_tmp_ent = sample[text_key][sample[ent_key][idx][0]:sample[ent_key][idx][1]]
        
        len_text = len((text_tmp_ent))
        tmp_ent[0] = ent_start
        tmp_ent[1] = tmp_ent[0] + len_text
        tmp_ent[2] = sample[ent_key][idx][2]
        shuffled_ents.append(tmp_ent)

        if len(blanks) > 0:
            next_blank = blanks.pop(0)
            ent_start += len((text_tmp_ent)) + next_blank[1] - next_blank[0]
        else:
            pass

        shuffled_text += text_tmp_ent + next_blank[2]
    sample.update({
        text_key: shuffled_text,
        ent_key: shuffled_ents,
    })
    if verbose:
        print_list = []
        for i in range(len(sample[f'idx_{shuffle_lang}'])):
            row = []
            for l in sample['sample_langs']:
                row.append([[sample[f'idx_{l}'].index(i)], sample[f'ents_{l}'][sample[f'idx_{l}'].index(i)] + [sample[f'text_{l}'][sample[f'ents_{l}'][sample[f'idx_{l}'].index(i)][0]:sample[f'ents_{l}'][sample[f'idx_{l}'].index(i)][1]]]])
            print_list.append(row)
        print(pd.DataFrame(print_list))
        print(sample['idx_en'])
        print(sample['idx_it'])

    return sample



